REMEMBER: FIRST CREATE A COPY OF THIS FILE WITH A UNIQUE NAME AND DO YOUR WORK THERE. AND MAKE SURE YOU COMMIT YOUR CHANGES TO THE `hw3_submissions` BRANCH.

# Assignment 3 | Cleaning and Exploring Data with Pandas



<img src="data/scoreCard.jpg" width=250>

In this assignment, you will investigate restaurant food safety scores for restaurants in San Francisco. Above is a sample score card for a restaurant. The scores and violation information have been made available by the San Francisco Department of Public Health. 

## Loading Food Safety Data


There are 2 files in the data directory:
1. business.csv containing food establishments in San Francisco
1. inspections.csv containing retaurant inspections records

Let's start by loading them into Pandas dataframes.  One of the files, business.csv, has encoding (ISO-8859-1), so you will need to account for that when reading it.

### Question 1

#### Question 1a
Read the two files noted above into two pandas dataframes named `bus` and `ins`, respectively. Print the first 5 rows of each to inspect them.


In [1]:
import pandas as pd

In [7]:
import os 
bus = pd.read_csv('businesses.csv', encoding='ISO-8859-1')
ins = pd.read_csv('inspections.csv')

In [8]:
bus.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494
3,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN
4,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051


In [9]:
ins.head()

,business_id,score,date,type
0,10,82,20160503,routine
1,10,94,20140729,routine
2,10,92,20140114,routine
3,19,94,20160513,routine
4,19,94,20141110,routine


## Examining the Business data

From its name alone, we expect the `businesses.csv` file to contain information about the restaurants. Let's investigate this dataset.

### Question 2

#### Question 2a: How many records are there?

In [10]:
print('How many records are there: ', len(bus))

How many records are there:  6315


#### Question 2b: How many unique business IDs are there?  

In [13]:
print('How many unique business IDs: ', len(bus.groupby('business_id')['business_id'].count()))

How many unique business IDs:  6315


#### Question 2c: What are the 5 most common businesses by name, and how many are there in San Francisco?

In [38]:
print("top five businesses are Starbucks, Peet's Coffee, Mcdonalds, Walgreens, and SF Soup Company")
top_5 = bus.groupby('name')['name'].count().to_frame('count').sort_values(by = 'count', ascending = False).head()
print(top_5)
top_5 = top_5.reset_index()['name'].to_list()
# How many are there in San Francisco? 
bus[(bus['name'].isin(top_5)) &(bus['city'] == 'San Francisco')].groupby('name')['name'].count()

top five businesses are Starbucks, Peet's Coffee, Mcdonalds, Walgreens, and SF Soup Company
                            count
name                             
STARBUCKS COFFEE               72
PEET'S COFFEE & TEA            24
MCDONALDS                      12
WALGREENS                      11
SAN FRANCISCO SOUP COMPANY     11


name
MCDONALDS                     12
PEET'S COFFEE & TEA           24
SAN FRANCISCO SOUP COMPANY    11
STARBUCKS COFFEE              72
WALGREENS                     11
Name: name, dtype: int64

## Zip code

Next, let's  explore some of the variables in the business table. We begin by examining the postal code.

### Question 3

#### Question 3a
How are the zip code values stored in python (i.e. data type)?

To answer this you might want to examine a particular entry.

In [43]:
bus['postal_code'].dtype
print('data type of zip codes is Object')

data type of zip codes is Object


#### Question 3b

What are the unique values of postal_code?

In [44]:
bus['postal_code'].unique()

array(['94104', '94109', '94133', '94110', '94122', '94103', '94115',
       '94131', '94111', '94117', '94107', '94108', '94102', '94132',
       '94105', '94134', '94124', '94116', '94121', '94118', '94112',
       '94123', '94114', '94127', '941102019', '941', '94545', '94066',
       '941033148', nan, '94158', '95105', '94013', '94130', 'CA',
       '92672', '94120', '94143', '94609', '94101', '00000', '94188',
       '94621', '94014', '94129', '94602', 'Ca'], dtype=object)

#### Question 3c

Let's say we decide to exclude the businesses that have no zipcode for our analysis (which might include food trucks for example).  Use the list of valid 5-digit zip codes below to create a new dataframe called bus_valid, with only businesses whose postal_codes show up in this list of valid zipcodes. How many businesses are there in this new dataframe?

In [93]:
validZip = ["94102", "94103", "94104", "94105", "94107", "94108",
            "94109", "94110", "94111", "94112", "94114", "94115",
            "94116", "94117", "94118", "94121", "94122", "94123", 
            "94124", "94127", "94131", "94132", "94133", "94134"]

In [94]:
vb = bus[bus['postal_code'].isin(validZip)]

## Latitude and Longitude

Another aspect of the data we want to consider is the prevalence of missing values. If many records have missing values then we might be concerned about whether the nonmissing values are representative of the population.

### Question 4
 
Consider the longitude and latitude in the business DataFrame. 

#### Question 4a

How many businesses are missing longitude values, working with only the businesses that are in the list of valid zipcodes?

In [95]:
print('number of null values for longitude column ;', len(vb[pd.isnull(vb['longitude'])]))

number of null values for longitude column ; 2483


#### Question 4b

Create a new dataframe with one row for each valid zipcode. The dataframe should include the following three columns:

1. `postal_code`: Contains the zip codes in the `validZip` variable above.
2. `null_lon`: The number of businesses in that zipcode with missing `longitude` values.
3. `not_null_lon`: The number of businesses without missing `longitude` values.

In [96]:
len(vb)
vb['null_lon'] = (vb['longitude'].isna())*1
missing_vb = vb.groupby(['postal_code', 'null_lon'])['postal_code'].count().to_frame()
missing_vb.head()

<ipython-input-96-c9a8b5069cc1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vb['null_lon'] = (vb['longitude'].isna())*1


postal_code
postal_code null_lon             
94102       0                 262
            1                 196
94103       0                 306
            1                 252
94104       0                  74

#### 4c. Do any zip codes appear to have more than their 'fair share' of missing longitude? 

To answer this, you will want to compute the proportion of missing longitude values for each zip code, and print the proportion missing longitude, and print the top five zipcodes in descending order of proportion missing postal_code.


In [128]:

missing_vb_1 = missing_vb.rename(columns = {'postal_code': 'postal code count'}).reset_index()
# sum across postal codes and divide entries by sum 
sum_per_code = missing_vb_1.groupby('postal_code')['postal code count'].sum()
missing_vb_rows = missing_vb_1[missing_vb_1['null_lon'] == 1]
# merge the two datasets
proportions = sum_per_code.to_frame().reset_index().merge(missing_vb_rows, left_on = 'postal_code', right_on = 'postal_code')
# create new column for proportion of missing longitudes
proportions['proportions missing'] = proportions['postal code count_y'] / proportions['postal code count_x']
proportions.loc[:,['postal_code', 'proportions missing']].sort_values(by = 'proportions missing', ascending = False)

,postal_code,proportions missing
4,94107,0.547672
18,94124,0.536585
21,94132,0.485075
10,94114,0.469565
16,94122,0.455556
1,94103,0.451613
2,94104,0.443609
3,94105,0.442478
7,94110,0.429060
0,94102,0.427948


# Investigate the inspection data

Let's now turn to the inspection DataFrame. Earlier, we found that `ins` has 4 columns, these are named `business_id`, `score`, `date` and `type`.  In this section, we determine the granularity of `ins` and investigate the kinds of information provided for the inspections. 

### Question 5

#### Question 5a
As with the business data, assess whether there is one inspection record for each business, by counting how many rows are in the data and how many unique businesses there are in the data. If they are exactly the same number, it means there is only one inspection per business, clearly.

In [132]:
print('unique rows in the data ;', len(ins['business_id'].unique()))

unique rows in the data ; 5730


In [134]:
print('length of dataset: ', len(ins['business_id']))

length of dataset:  15430


#### Question 5b

What values does `type` take on? How many occurrences of each value is in the DataFrame? Create a new dataframe named `ins2` by copying `ins` and keeping only records with values of `type` that occur more than 10 times in the original table. In other words, eliminate records that have values of `type` that occur rarely (< 10 times). Check the result to make sure rare types are eliminated.

In [162]:
ins['type'].dtype
ins2 = ins.copy()
ins2.groupby('type')['type'].count()

type
complaint        1
routine      15429
Name: type, dtype: int64

In [163]:
ins2 = ins2[ins2['type'] != 'complaint']
ins2.head()

,business_id,score,date,type
0,10,82,20160503,routine
1,10,94,20140729,routine
2,10,92,20140114,routine
3,19,94,20160513,routine
4,19,94,20141110,routine


#### Question 5c

Since the data was stored in a .csv file, the dates are formatted as strings such as `20160503`. Once we read in the data, we would like to have dates in an appropriate format for analysis. Add a new column called `year` by capturing the first four characters of the date column. 

Hint: we have seen multiple ways of doing this in class, includings `str` operations, `lambda` functions, `datetime` operations, and others. Choose the method that works best for you :)

In [165]:
from datetime import datetime
ins2['date'] = ins2['date'].apply(str)
ins2['year'] = ins2['date'].apply(lambda x: x[0:4])
ins2.head()

,business_id,score,date,type,year
0,10,82,20160503,routine,2016
1,10,94,20140729,routine,2014
2,10,92,20140114,routine,2014
3,19,94,20160513,routine,2016
4,19,94,20141110,routine,2014


#### Question 5d

What range of years is covered in this data set? Are there roughly same number of inspections each year? Try dropping records for any years with less than 50 inspections and store the result in a new dataframe named `ins3`.

In [166]:
# range of years 
ins2['year'].unique()

array(['2016', '2014', '2015', '2013'], dtype=object)

In [168]:
ins2.groupby(['year'])['year'].count()

year
2013      38
2014    5629
2015    3923
2016    5839
Name: year, dtype: int64

In [171]:
# drop years with less than 50 entries 
ins2 = ins2[ins2['year'] != '2013']
ins2.head()

,business_id,score,date,type,year
0,10,82,20160503,routine,2016
1,10,94,20140729,routine,2014
2,10,92,20140114,routine,2014
3,19,94,20160513,routine,2016
4,19,94,20141110,routine,2014


Let's examine only the inspections for one year: 2016. This puts businesses on a more equal footing because [inspection guidelines](https://www.sfdph.org/dph/eh/Food/Inspections.asp) generally refer to how many inspections should occur in a given year.

In [172]:
ins2_2016 = ins2[ins2['year'] != '2016']

### Question 6

#### Question 6a

Merge the business and 2016 inspections data, keeping all businesses regardless of whether they show up in the inspections file. Show the first several rows of the resulting dataframe.

In [177]:
merged_dataset = vb.merge(ins2_2016, left_on = 'business_id', right_on = 'business_id' )

#### Question 6b
Print the 20 lowest rated businesses names, their addresses, and their ratings.

In [181]:
merged_dataset.sort_values(by = 'score', ascending = True).head(20)

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number,null_lon,score,date,type,year
6836,68633,ABC BAKERY CAFE,650 JACKSON ST,San Francisco,CA,94133,37.796133,-122.406237,+14159810803,0,46,20140512,routine,2014
4107,18480,IMPERIAL PALACE,818 WASHINGTON ST,San Francisco,CA,94108,37.795138,-122.406863,+14153778715,0,47,20141023,routine,2014
5980,64154,"YUMMY DIM SUM & FAST FOOD, LLC",930 STOCKTON ST,San Francisco,CA,94108,37.794457,-122.407896,+14159862783,0,50,20140507,routine,2014
1598,3123,LES JOULINS,44 ELLIS ST,San Francisco,CA,94102,NaN,NaN,+14153975397,1,53,20140522,routine,2014
1583,3115,YEE'S RESTAURANT,1131 GRANT AVE,San Francisco,CA,94133,37.797292,-122.406914,NaN,0,53,20140624,routine,2014
5553,61436,OZONE THAI RESTAURANT AND LOUNGE,598 02ND ST,San Francisco,CA,94107,37.781947,-122.392239,+14155363266,0,54,20150706,routine,2015
3839,10102,BODEGA BISTRO,607 LARKIN ST,San Francisco,CA,94109,37.783533,-122.417603,+14159211218,0,54,20150914,routine,2015
4329,25572,BRISTOL FARMS #2317,845 MARKET ST 010,San Francisco,CA,94103,37.784848,-122.406892,+14159790106,0,54,20150427,routine,2015
7696,74084,MISS SAIGON LLC,100 06TH ST,San Francisco,CA,94103,NaN,NaN,+14155220332,1,54,20141217,routine,2014
1772,3459,BASIL THAI RESTAURANT & BAR,1175 FOLSOM ST,San Francisco,CA,94103,37.775508,-122.409361,+14155528999,0,54,20150407,routine,2015


## Done!

Now commit this notebook to your `hw3_submissions` branch, push it to your GitHub repo, and open a PR!